In [217]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast

In [218]:
#Importing data
df = pd.read_csv('recipe_train.csv')

# Introduction
When building machine learning models, we require a baseline model to act as reference to help gauge the performance of our models. The baseline models usually are simple models that have little predictive powers, but serves as a great benchmark.

The baseline models that we will be using as our benchmark is recommending items based on the popularity. This will be based on the higest average rating with atleast a certain number of reviews.

# Recommender system by Popularity of item

In [219]:
df.head(5)

,Unnamed: 0,RecipeId,RecipeName,RecipeCategory,Keywords,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,...,ingredients,ingredients_raw_str,minutes,n_steps,n_ingredients,AggregatedRating,ReviewCount,year,month,day_of_week
0,4240,18795,Blueberry Cream Cheesecake,Cheesecake,['Dessert' 'Grains' 'Cheese' 'Berries' 'Fruit'...,533.6,30.3,15.3,120.8,220.8,...,"['graham wafer crumbs', 'vanilla wafer crumbs'...","["""",""3/4 cup graham wafer crumbs"",""1/2 cu...",95,28,16,3.00,1.0,2002,2,6
1,31039,79029,Horseradish Beef Roll ups,Cheese,['Meat' 'Potluck' 'Christmas' 'Thanksgiving' '...,209.1,8.2,3.9,36.8,395.6,...,"['fat free cream cheese', 'prepared horseradis...","[""1 (8 ounce) package fat-free cream chees...",250,9,7,4.88,8.0,2003,12,1
2,93789,252366,Holiday Snack Mix Taste of Home,Christmas,['< 60 Mins' 'Easy'],1396.8,57.3,20.9,47.3,2124.4,...,"['pretzel twists', 'ritz crackers', 'rice chex...","[""11 ounces little pretzel twists"",""10 1/2...",55,7,11,5.00,2.0,2007,9,1
3,121013,360384,Kellibou Guacamole,Lunch/Snacks,['Vegetable' 'Mexican' '< 30 Mins' 'For Large ...,56.6,4.9,0.7,0.0,2.9,...,"['avocado', 'ground cumin', 'roma tomato', 'sw...","[""4 avocados, peeled, and seeded "",""1 ...",22,6,8,4.67,3.0,2009,3,2
4,49950,122617,Pamela's Chocolate Biscuit Dessert,Breads,['Dessert' '< 30 Mins' 'Easy'],120.1,5.5,1.5,1.0,197.3,...,"['biscuit', 'chocolate chips', 'cinnamon sugar...","[""1 (6 ounce) can biscuits"","" chocolate c...",17,7,5,4.50,2.0,2005,5,1


In [220]:
#Drop Unnamed: 0 column
df.drop(columns = 'Unnamed: 0', inplace = True)

In [221]:
#Taking top 10 highest rated recipes
df.sort_values('AggregatedRating', ascending = False).head(10)

,RecipeId,RecipeName,RecipeCategory,Keywords,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,...,ingredients,ingredients_raw_str,minutes,n_steps,n_ingredients,AggregatedRating,ReviewCount,year,month,day_of_week
73876,84267,Cauliflower with Chicken,One Dish Meal,['Chicken' 'Poultry' 'Cauliflower' 'Vegetable'...,125.0,7.3,1.0,0.9,695.0,11.1,...,"['cauliflower', 'chicken meat', 'egg whites', ...","[""1 whole cauliflower"",""1 cup chopped...",25,9,7,5.0,1.0,2004,2,1
108794,95568,Toasty Cheese Ravioli,< 15 Mins,['Beginner Cook' 'Easy'],129.4,11.9,7.4,30.5,131.3,5.0,...,"['cheese ravioli', 'butter', 'italian style br...","[""1 lb cheese ravioli (fresh \""or\"" froze...",10,6,4,5.0,1.0,2004,7,1
108811,336922,Irish Tea Bread,Quick Breads,['Breads' 'Fruit' 'Australian' 'European' 'Bru...,434.2,1.7,0.4,42.3,667.6,92.0,...,"['dried mixed fruit', 'tea', 'brandy', 'dark b...","[""475 g dried mixed fruit"",""300 ml ho...",780,10,7,5.0,1.0,2008,11,3
48995,403390,Buffalo Chicken Dip,Very Low Carbs,['Weeknight' '< 60 Mins'],453.9,41.3,15.2,95.6,748.7,3.5,...,"['chicken breasts', 'philadelphia cream cheese...","[""4 chicken breasts"",""2 (8 ounce) pack...",40,8,6,5.0,1.0,2009,12,4
108810,75813,Fantastic Marshmallow Fluff Clone,Low Protein,['Low Cholesterol' 'Healthy' '< 15 Mins' 'Easy'],1476.3,0.8,0.0,0.0,878.0,382.8,...,"['egg whites', 'light corn syrup', 'salt', 'ic...","[""3 egg whites"",""2 cups light corn ...",10,6,5,5.0,2.0,2003,11,3
108808,100127,Leaf Salad,Lunch/Snacks,['Spinach' 'Greens' 'Vegetable' 'Low Protein' ...,155.6,7.9,1.2,6.1,239.1,22.3,...,"['iceberg lettuce', 'green cabbage', 'red cabb...","[""3 cups iceberg lettuce"",""2 cups gr...",30,8,9,5.0,1.0,2004,9,0
48998,56218,Adi's Shepherd's Pie,Savory Pies,['One Dish Meal' 'Potato' 'Corn' 'Vegetable' '...,641.2,28.2,9.9,138.1,961.5,60.1,...,"['ground beef', 'egg', 'breadcrumb', 'salt', '...","[""2 lbs ground beef"",""1 egg, beaten...",75,9,13,5.0,1.0,2003,3,2
108806,374998,Mujadara,< 4 Hours,[None],265.0,9.8,1.4,0.0,393.4,34.8,...,"['olive oil', 'sweet onions', 'green lentil', ...","[""1/4 cup olive oil"",""2 medium sweet ...",120,8,6,5.0,1.0,2009,5,4
49000,290078,Very Berry Blueberry Muffins,Quick Breads,['Breads' 'Berries' 'Fruit' 'Mixer' '< 60 Mins...,164.6,1.5,0.4,47.4,301.9,32.7,...,"['all purpose flour', 'baking powder', 'salt',...","[""2 cups all-purpose flour"",""2 teaspoon...",32,14,10,5.0,1.0,2008,3,1
108805,195160,Smoked and Fresh Salmon Rillettes,Spreads,['Canadian' 'Savory' 'Brunch' '< 30 Mins'],280.9,18.2,8.9,67.0,195.8,11.2,...,"['olive oil', 'boneless salmon fillet', 'smoke...","[""1 tablespoon olive oil"",""1 lb fresh...",30,9,10,5.0,2.0,2006,11,6


### Observations
There are a few issues when we try to filter the top 10 recipes by AggregatedRating. There are a few recipes with 6 or less Review Counts. In order for us to recommend the most popular items, we need to take into account both the rating and review count. As such, we need to implement a threshold of reviews prior to filtering the top 10 recipes by rating.

In [222]:
threshold = 50
df_reviews_filter = df[df['ReviewCount'] >= threshold]

df_popular_recipe = df_reviews_filter.sort_values('AggregatedRating', ascending = False)
df_popular_recipe.head(10)

,RecipeId,RecipeName,RecipeCategory,Keywords,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,...,ingredients,ingredients_raw_str,minutes,n_steps,n_ingredients,AggregatedRating,ReviewCount,year,month,day_of_week
117758,98635,Cantaberry Crumble Top Muffins,Quick Breads,['Breads' 'Strawberry' 'Berries' 'Melons' 'Fru...,333.3,12.7,1.8,0.0,499.1,51.0,...,"['cinnamon', 'margarine', 'sugar', 'unbleached...","["""",""1/4 teaspoon cinnamon"",""1 1/2 tables...",30,13,16,5.00,52.0,2004,8,4
34424,112837,Chicken and Fresh Grapefruit Stir-Fry,Citrus,['Fruit' 'Meat' 'Asian' '< 15 Mins' 'Stir Fry'],156.3,2.7,0.3,34.2,41.6,18.5,...,"['grapefruit', 'pineapple chunks', 'cornstarch...","[""1 grapefruit, peeled "",""1 (8 ounce) ...",15,14,8,4.96,74.0,2005,3,0
146288,39186,Chicken Bang Bang,Lunch/Snacks,['Chicken' 'Poultry' 'Meat' 'Asian' 'Kosher' '...,558.5,28.6,7.2,109.0,765.9,31.1,...,"['chicken breasts', 'rice noodles', 'garlic', ...","[""1 1/2 lbs poached chicken breasts"",""3 o...",20,7,11,4.96,55.0,2002,9,1
64984,43375,Lenguas De Gato (Cat's Tongue),Dessert,['Cookie & Brownie' 'Filipino' 'Asian' '< 30 M...,55.0,2.6,1.6,6.8,37.2,7.3,...,"['butter', 'sugar', 'egg white', 'flour', 'van...","[""1/2 cup butter or 1/2 cup margarine""...",20,8,6,4.96,50.0,2002,10,2
14861,75659,Apple Filling for Pies,Pie,['Dessert' 'Apple' 'Fruit' 'Low Protein' 'Low ...,105.2,0.2,0.1,0.0,2.3,27.1,...,"['apple', 'lemon juice', 'sugar', 'cornstarch'...","[""4 cups diced apples"",""2 tablespoons ...",45,7,7,4.96,57.0,2003,11,2
109041,106145,Chicken Crepes With Creamy Tarragon Sauce,Chicken Breast,['Chicken' 'Poultry' 'Meat' 'European' '< 60 M...,808.8,32.3,15.2,292.5,983.8,54.0,...,"['boneless skinless chicken breasts', 'frozen ...","["""",""16 ounces boneless skinless chicken ...",45,21,13,4.95,63.0,2004,12,2
43823,67318,Classic Italian Minestrone (Soupe Paysanne),Greens,['Vegetable' 'European' 'Weeknight' 'For Large...,128.0,4.2,2.0,4.3,274.7,18.9,...,"['pork fat', 'boiling potatoes', 'green cabbag...","[""1/4 lb pork fat, chopped "",""2 lbs b...",150,5,14,4.94,67.0,2003,7,3
41495,78032,Southwest Potato Lentil Stew,Stew,['One Dish Meal' 'Potato' 'Lentil' 'Beans' 'Ve...,160.1,0.6,0.1,0.0,385.5,35.2,...,"['lentil', 'potato', 'carrot', 'onion', 'celer...","[""1/2 cup lentils, uncooked "",""2 medium ...",60,4,12,4.93,54.0,2003,12,4
2108,383593,Holy Crap These are Amazing Cookies,Drop Cookies,['Dessert' 'Cookie & Brownie' 'Sweet' 'Weeknig...,157.7,8.4,2.3,0.0,99.6,19.6,...,"['sugar', 'corn syrup', 'peanut butter', 'corn...","[""1 cup sugar"",""1 cup corn syrup"",""1...",15,11,5,4.93,69.0,2009,7,4
130739,257354,Amaretto Spiked Chocolate Mousse (Vegan),Dessert,['Soy/Tofu' 'Beans' 'Vegan' 'Toddler Friendly'...,41.7,1.9,0.2,0.0,5.3,2.9,...,"['chocolate soymilk', 'semisweet vegan chocola...","[""1/2 cup chocolate soymilk"",""1 (9 ounce...",13,13,5,4.92,61.0,2007,10,5


## Creating a function for popularity recommender system

In [223]:
def recommender_popularity(data,n_items,threshold):
    """
    
    data: input data recommender system will be running on
    n_items: number of items to return from recommender system
    threshold: minimum number of review count for recommender model
    
    """
    n = n_items
    threshold = threshold
    
    df_threshold = data[data['ReviewCount'] >= threshold]
    
    df_popular_recipe = df_threshold.sort_values('AggregatedRating', ascending = False)
    
    return df_popular_recipe.head(10)

In [224]:
#Test
df_recommender = recommender_popularity(df,10,100)
df_recommender

,RecipeId,RecipeName,RecipeCategory,Keywords,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,...,ingredients,ingredients_raw_str,minutes,n_steps,n_ingredients,AggregatedRating,ReviewCount,year,month,day_of_week
86795,83575,All American Jambalaya,One Dish Meal,['Rice' 'Meat' 'Creole' '< 30 Mins'],319.0,15.7,7.0,61.6,474.4,26.8,...,"['butter', 'ground beef', 'onion', 'green pepp...","[""2 tablespoons butter"",""1 lb ground...",30,9,9,4.92,156.0,2004,2,1
104381,77369,The Best Sugar Cookie Cut-Outs W/ Icing,Dessert,['Cookie & Brownie' 'Kid Friendly' 'Christmas'...,1345.3,37.2,22.0,195.5,618.3,230.1,...,"['egg', 'butter', 'sugar', 'evaporated milk', ...","["""",""3 eggs"",""1 cup butter"",""2 cu...",32,13,11,4.92,119.0,2003,11,2
65590,24087,Corn Bisque with Red Bell Pepper and Rosemary,Corn,['Vegetable' 'Low Protein' '< 60 Mins' 'Stove ...,476.2,29.8,17.5,77.3,450.1,47.9,...,"['butter', 'onion', 'carrot', 'celery', 'froze...","[""4 (1/2 stick) tablespoons butter"",""2 cu...",50,14,10,4.91,122.0,2002,3,6
22185,166742,Skewered Grilled Red Potatoes,< 60 Mins,[None],113.8,0.3,0.1,0.0,41.5,24.8,...,"['red potatoes', 'water', 'miracle whip', 'chi...","[""2 lbs red potatoes (quartered or cut in...",33,9,7,4.87,134.0,2006,5,1
136325,154043,Pondering Parsnips (Those Really Ugly Carrots),Vegetable,['Low Protein' 'Vegan' 'Low Cholesterol' 'Heal...,133.9,2.7,0.4,0.0,15.6,27.3,...,"['parsnip', 'water', 'extra virgin olive oil',...","[""2 lbs fresh parsnips, small-medium sized...",22,8,7,4.87,107.0,2006,1,1
101527,183409,Wedding Cake Fudge,Candy,['Dessert' 'Low Protein' 'Healthy' 'Christmas'...,162.2,2.5,1.5,7.3,105.3,35.5,...,"['unsalted butter', 'sugar', 'half and half', ...","[""1 tablespoon unsalted butter"",""2 cups...",120,9,7,4.85,141.0,2006,8,0
124203,488265,Best Ever Pumpkin Bread,Breads,['Thanksgiving' '< 60 Mins' 'For Large Groups'...,305.3,12.1,1.8,37.2,359.6,46.6,...,"['white sugar', 'cooking oil', 'egg', 'all pur...","[""2 1/2 cups white sugar"",""1 cup cook...",50,18,8,4.85,132.0,2012,10,6
54079,91080,Honey Spice Cookies,Dessert,['Cookie & Brownie' '< 30 Mins' 'For Large Gro...,142.5,6.1,3.7,23.0,183.7,21.0,...,"['egg', 'honey', 'flour', 'salt', 'brown sugar...","[""1 egg"",""1/4 cup honey"",""2 1/4 cup...",17,7,10,4.84,452.0,2004,5,2
93333,192039,5 Min Ice Cream,Frozen Desserts,['Dessert' 'Low Protein' 'Low Cholesterol' '< ...,105.5,4.5,2.8,17.1,18903.3,12.1,...,"['ziploc bag', 'ziploc bag', 'milk', 'sugar', ...","[""1 gallon ziploc bag"",""1 quart zipl...",7,6,6,4.84,122.0,2006,10,1
8921,131158,Simple Cincinnati Chili,Spaghetti,['Beans' 'Meat' 'Spicy' '< 30 Mins'],450.7,18.4,6.9,77.1,951.6,39.3,...,"['ground beef', 'chili seasoning mix', 'ground...","[""1 lb ground beef"",""1 (1 1/4 ounce) pac...",30,7,8,4.84,135.0,2005,7,0


# Model Evaluation
To provide a baseline metric, we need to look at the evaluation metrics for our popularity recommender model.<br>

The goal of our project is to provide our users with a grocery list based on recipes that have same ingredients as the input. The metric best suited to determine this is how relevant the recipes are to the user. Metrics we will use for this will include precision and recall on the top-N items suggested to users. Accuracy would be more focused on when we build our Collaborative model.

To determine whether an item is a true positive to a user, we need to determine how many of the ingredients in our input matches the ingredients listed. Then we need to binarize the outcomes so that we can compute the precision and recall score at k (k is the number of top-N items listed).

In [225]:
#Sample input we will use to test precision and recall score.
input_ingredients = ['egg','carrot','onion','garlic']

In [226]:
df_recommender['ingredients'] = df_recommender['ingredients'].apply(lambda x: list(ast.literal_eval(x)))
df_recommender.reset_index(drop = True, inplace = True)

In [227]:
range(len(df_recommender))

range(0, 10)

In [236]:
#Including percentage calculation of ingredients that are same in input vs recommended recipe
for i in range(len(df_recommender)):
    ingredient_list = df_recommender.loc[i, 'ingredients'] 
    intersection = len(set(input_ingredients).intersection(set(ingredient_list)))
    union = len(set(input_ingredients).union(set(ingredient_list)))
    jaccard_similarity = intersection / union
    df_recommender.loc[i, 'Jaccard_similarity'] = jaccard_similarity
    df_recommender.loc[i, 'True_recipe'] = 1 if jaccard_similarity > 0 else 0
    

In [237]:
df_recommender = df_recommender[['RecipeName','ingredients','AggregatedRating','ReviewCount','Jaccard_similarity','True_recipe']]

In [238]:
df_recommender 

,RecipeName,ingredients,AggregatedRating,ReviewCount,Jaccard_similarity,True_recipe
0,All American Jambalaya,"[butter, ground beef, onion, green pepper, tom...",4.92,156.0,0.083333,1.0
1,The Best Sugar Cookie Cut-Outs W/ Icing,"[egg, butter, sugar, evaporated milk, baking s...",4.92,119.0,0.071429,1.0
2,Corn Bisque with Red Bell Pepper and Rosemary,"[butter, onion, carrot, celery, frozen corn ke...",4.91,122.0,0.166667,1.0
3,Skewered Grilled Red Potatoes,"[red potatoes, water, miracle whip, chicken br...",4.87,134.0,0.000000,0.0
4,Pondering Parsnips (Those Really Ugly Carrots),"[parsnip, water, extra virgin olive oil, salt,...",4.87,107.0,0.000000,0.0
5,Wedding Cake Fudge,"[unsalted butter, sugar, half and half, whole ...",4.85,141.0,0.000000,0.0
6,Best Ever Pumpkin Bread,"[white sugar, cooking oil, egg, all purpose fl...",4.85,132.0,0.090909,1.0
7,Honey Spice Cookies,"[egg, honey, flour, salt, brown sugar, ground ...",4.84,452.0,0.076923,1.0
8,5 Min Ice Cream,"[ziploc bag, ziploc bag, milk, sugar, vanilla,...",4.84,122.0,0.000000,0.0
9,Simple Cincinnati Chili,"[ground beef, chili seasoning mix, ground cinn...",4.84,135.0,0.090909,1.0


In [239]:
df_recommender.loc[9,'ingredients']

['ground beef',
 'chili seasoning mix',
 'ground cinnamon',
 'tomato sauce',
 'kidney beans',
 'cooked spaghetti',
 'cheddar cheese',
 'onion']

In [240]:
#Precision score
df_recommender['True_recipe'].sum()/df_recommender['True_recipe'].count()

0.6

### Conclusion
Popularity precision score is 60%. This is our baseline score that we will need to improve against for our other models. <br>

**Note of Caution**
We defined True Positive in our precision score as a recipe that had a jaccard score greater than 0 meaning that the recipe had atleast one item from input. We need to take into fact in our other models the affects on the similarity score and its improvement.